<a href="https://colab.research.google.com/github/abhishek1472/DevClubAssignment1/blob/master/Skill_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
upload=files.upload()

Saving train-data.csv to train-data.csv


In [ ]:
import numpy as np
import pandas as pd
datainput = pd.read_csv('train-data.csv')
data=datainput[datainput['auto_label'].isin([1,2])]
print(data)

      skill_id                                         skill_name  auto_label
0            1                                     able to listen         2.0
1            2                                    accept feedback         2.0
2            3                                          adaptable         2.0
3            4                                     artistic sense         2.0
4            5                                          assertive         2.0
...        ...                                                ...         ...
86112    91504                 developing full-stack applications         1.0
86124    91702                                            onenote         1.0
86125    91949  cctv and voice infrastructure includince telep...         1.0
86126    92183   able to stand for long hours in cold environment         1.0
86127    92185                                 customer awareness         1.0

[15541 rows x 3 columns]


In [ ]:
data_arr=data.values
np.random.shuffle(data_arr)

In [ ]:
skills=data_arr[:,1]
labels=data_arr[:,2].astype('float64')
for i in range(len(skills)):
  skills[i]=str(skills[i])
print(skills.shape)

(15541,)


In [ ]:

# from keras.models import Sequential
# from keras.layers.experimental.preprocessing import TextVectorization
# vectorize_layer=TextVectorization(output_mode='tf-idf')
# vectorize_layer.adapt(skills)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vectorizor = TfidfVectorizer(stop_words = 'english',max_features = 20000,ngram_range=(1, 1))
vectorizer=tf_idf_vectorizor.fit_transform(skills)

In [ ]:
from keras.preprocessing.text import Tokenizer
t=Tokenizer()
t.fit_on_texts(skills)
vectorizer=t.texts_to_matrix(skills,mode='tfidf')
print(vectorizer.shape)

(15541, 7523)


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
vectorizer_norm = normalize(vectorizer)
sklearn_pca = PCA(n_components = 500)
Y_sklearn = sklearn_pca.fit_transform(vectorizer_norm)

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# X=Y_sklearn[:2000]
# Y=labels-1
# Y=Y[:2000]
# def create_model():
#   model=Sequential()
#   model.add(Dense(64,input_dim=X.shape[1],activation='relu'))
#   model.add(Dense(1,activation='sigmoid'))
#   model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#   return model
# estimator = KerasClassifier(build_fn=create_model, epochs=100, batch_size=5, verbose=0)
# kfold = StratifiedKFold(n_splits=10, shuffle=True)
# results = cross_val_score(estimator, X, Y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
X_train=Y_sklearn[:12500]
X_val=Y_sklearn[12500:]
Y_train=labels[:12500]-1
Y_val=labels[12500:]-1
print(type(X_train))
print(Y_val)

<class 'numpy.ndarray'>
[0. 0. 1. ... 0. 0. 0.]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model1=Sequential()
model1.add(Dense(64,input_dim=X_train.shape[1],activation='relu'))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model1.fit(X_train, Y_train,batch_size=200,epochs=100,verbose=1,
                    validation_data=(X_val,Y_val))


Epoch 1/100
63/63 [==============================] - 1s 9ms/step - loss: 0.6394 - accuracy: 0.8792 - val_loss: 0.4634 - val_accuracy: 0.9648
Epoch 2/100
63/63 [==============================] - 0s 3ms/step - loss: 0.4056 - accuracy: 0.9612 - val_loss: 0.2476 - val_accuracy: 0.9658
Epoch 3/100
63/63 [==============================] - 0s 3ms/step - loss: 0.2202 - accuracy: 0.9614 - val_loss: 0.1415 - val_accuracy: 0.9697
Epoch 4/100
63/63 [==============================] - 0s 4ms/step - loss: 0.1338 - accuracy: 0.9691 - val_loss: 0.0988 - val_accuracy: 0.9783
Epoch 5/100
63/63 [==============================] - 0s 3ms/step - loss: 0.1015 - accuracy: 0.9754 - val_loss: 0.0791 - val_accuracy: 0.9819
Epoch 6/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0824 - accuracy: 0.9821 - val_loss: 0.0691 - val_accuracy: 0.9836
Epoch 7/100
63/63 [==============================] - 0s 3ms/step - loss: 0.0814 - accuracy: 0.9813 - val_loss: 0.0640 - val_accuracy: 0.9845
Epoch 8/100
6

In [ ]:
s=["diversity awareness"]
v=t.texts_to_matrix(s,mode='tfidf')
v_norm = normalize(v)
v_pca = sklearn_pca.transform(v_norm)
y=model1.predict_classes(v_pca)
print(y)

[[0]]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
def predict(s):
  v=t.texts_to_matrix(s,mode='tfidf')
  v_norm = normalize(v)
  v_pca = sklearn_pca.transform(v_norm)
  y=model1.predict_classes(v_pca)
  return y
